<a href="https://colab.research.google.com/github/VitalyGladyshev/DevOps/blob/main/HW1_designapi_%D0%93%D0%BB%D0%B0%D0%B4%D1%8B%D1%88%D0%B5%D0%B2_%D0%92%D0%B8%D1%82%D0%B0%D0%BB%D0%B8%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HW1. Проектирование API
Автор: Гладышев Виталий Владимирович, 25.10.2025

Это задание выполняется в рамках модуля 1 «Проектирование API». Вы закрепите навыки разработки API, используя подход сode-first, затем будете придерживаться подхода API-first

> Чтобы получить максимальный балл, убедитесь, что ваш ноутбук запускается с нуля, структура понятна, а в выводах вы объясняете свои решения.  

## Подготовка окружения

In [1]:
!pip install fastapi[all] uvicorn["standard"]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.7/517.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.8/456.8 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 951.9/951.9 kB 50.1 MB/s eta 0:00:00


In [5]:
# !pip install nest-asyncio

Теперь, когда у нас установлены необходимые библиотеки, мы можем приступить к созданию нашего первого приложения FastAPI.

In [1]:
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from typing import Dict, Optional

In [2]:
import uvicorn
import nest_asyncio
import threading
import time
import requests
import json
import random

### Задание 1

Вам необходимо самостоятельно создать веб-сервер на основе FastAPI.

Задача: создать файл `main.py`, который будет содержать наш код API с четырьмя методами HTTP.



In [3]:
app = FastAPI()

sample_db: Dict[int, dict] = {}

# объект для обмена
class Item(BaseModel):
    name: str
    description: Optional[str] = None

# POST
@app.post("/items/")
async def create_item(item: Item) -> dict:
    item_id = len(sample_db) + 1
    sample_db[item_id] = item.model_dump()
    # print(f"item.model_dump(): {item.model_dump()}")
    return {"id": item_id, **item.model_dump()}

# GET
@app.get("/items/{item_id}")
async def read_item(item_id: int) -> dict:
    if item_id not in sample_db:
        raise HTTPException(status_code=404, detail="Item отсутствует!")
    return {"id": item_id, **sample_db[item_id]}

# PUT
@app.put("/items/{item_id}")
async def update_item(item_id: int, item: Item) -> dict:
    if item_id not in sample_db:
        raise HTTPException(status_code=404, detail="Item не найден!")
    sample_db[item_id] = item.model_dump()
    return {"id": item_id, **sample_db[item_id]}

# DELETE
@app.delete("/items/{item_id}")
async def delete_item(item_id: int) -> dict:
    if item_id not in sample_db:
        raise HTTPException(status_code=404, detail="Item не найден!")
    deleted_item = sample_db.pop(item_id)
    return {"deleted": {"id": item_id, **deleted_item}}

In [4]:
nest_asyncio.apply()

item_counter = 0

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000)

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(3)

base_url = "http://localhost:8000"

print(f"1. POST")
new_item = {
    "name": "Тестовый объект",
    "description": "Это тестовое описание"
}

response = requests.post(f"{base_url}/items/", json=new_item)
print(f"Статус: {response.status_code}")
print(f"Ответ: {json.dumps(response.json(), indent=2, ensure_ascii=False)}")

print(f"\n2. GET")
response = requests.get(f"{base_url}/items/1")
print(f"Статус: {response.status_code}")
print(f"Ответ: {json.dumps(response.json(), indent=2, ensure_ascii=False)}")

print(f"\n3. PUT")
updated_item = {
    "name": "Обновленный объект",
    "description": "Обновленное описание"
}

response = requests.put(f"{base_url}/items/1", json=updated_item)
print(f"Статус: {response.status_code}")
print(f"Ответ: {json.dumps(response.json(), indent=2, ensure_ascii=False)}")

print(f"\n4. DELETE")
response = requests.delete(f"{base_url}/items/1")
print(f"Статус: {response.status_code}")
print(f"Ответ: {json.dumps(response.json(), indent=2, ensure_ascii=False)}")

INFO:     Started server process [60831]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


1. POST
INFO:     127.0.0.1:40572 - "POST /items/ HTTP/1.1" 200 OK
Статус: 200
Ответ: {
  "id": 1,
  "name": "Тестовый объект",
  "description": "Это тестовое описание"
}

2. GET
INFO:     127.0.0.1:40582 - "GET /items/1 HTTP/1.1" 200 OK
Статус: 200
Ответ: {
  "id": 1,
  "name": "Тестовый объект",
  "description": "Это тестовое описание"
}

3. PUT
INFO:     127.0.0.1:40594 - "PUT /items/1 HTTP/1.1" 200 OK
Статус: 200
Ответ: {
  "id": 1,
  "name": "Обновленный объект",
  "description": "Обновленное описание"
}

4. DELETE
INFO:     127.0.0.1:40604 - "DELETE /items/1 HTTP/1.1" 200 OK
Статус: 200
Ответ: {
  "deleted": {
    "id": 1,
    "name": "Обновленный объект",
    "description": "Обновленное описание"
  }
}


### Задание 2

Сервер должен отвечать валидным JSON на эндпоинте /json_data.

Задача: создать новый эндпоинт /json_data и подключить компонент JSONResponse на этом эндпоинте. Содержание JSONa не важно, главное, чтобы он был валидным.


In [5]:
sample_db: Dict[int, dict] = {}

@app.get("/json_data")
async def get_json_data() -> JSONResponse:
    json_data = {
        "status": "success",
        "message": "Данные успешно получены",
        "timestamp": time.time(),
        "data": {
            "users": [
                {"id": 1, "name": "Алексей", "role": "admin"},
                {"id": 2, "name": "Мария", "role": "user"},
                {"id": 3, "name": "Иван", "role": "user"}
            ]
        }
    }

    return JSONResponse(
        content=json_data,
        status_code=200,
        headers={"Content-Type": "application/json", "X-Custom-Header": "test"}
    )

In [6]:
nest_asyncio.apply()

item_counter = 0

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8001)

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(2)

base_url = "http://localhost:8001"

response = requests.get(f"{base_url}/json_data")
print(f"Статус: {response.status_code}")
print(f"Заголовки: {dict(response.headers)}")
json_response = response.json()
print(f"{json.dumps(json_response, indent=2, ensure_ascii=False)}")

INFO:     Started server process [60831]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)


INFO:     127.0.0.1:59922 - "GET /json_data HTTP/1.1" 200 OK
Статус: 200
Заголовки: {'date': 'Sat, 25 Oct 2025 17:00:03 GMT', 'server': 'uvicorn', 'content-type': 'application/json', 'x-custom-header': 'test', 'content-length': '259'}
{
  "status": "success",
  "message": "Данные успешно получены",
  "timestamp": 1761411604.8442042,
  "data": {
    "users": [
      {
        "id": 1,
        "name": "Алексей",
        "role": "admin"
      },
      {
        "id": 2,
        "name": "Мария",
        "role": "user"
      },
      {
        "id": 3,
        "name": "Иван",
        "role": "user"
      }
    ]
  }
}


### Задание 3

Обработка ошибок с использованием простого HTTPException.

Задачи:
1. Изучите стандартные коды ошибок.
2. Выберите любое случайное число в диапазоне 400—526.

> Чтобы возвращать ошибки с соответствующими HTTP-статусами вам нужно:
> 1. подключить класс `HTTPException` и просто выдать любой [код ошибки](https://ru.wikipedia.org/wiki/%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%BA%D0%BE%D0%B4%D0%BE%D0%B2_%D1%81%D0%BE%D1%81%D1%82%D0%BE%D1%8F%D0%BD%D0%B8%D1%8F_HTTP) (от 400 до 526);
> 2. создать маршрут /error, который будет генерировать ошибку с этим кодом в ответ на любой запрос.

В комментариях напишите, как вы понимаете, что означает выбранный вами код ошибки.

In [7]:
import random;print("используйте и опишите код ошибки", random.randint(400,526))

используйте и опишите код ошибки 445


#### Стандартные коды ошибок
- 400 Bad Request — сервер не понял запрос из-за неверного синтаксиса.
- 401 Unauthorized — для доступа к ресурсу требуется аутентификация.
- 403 Forbidden — сервер понял запрос, но отказывается его авторизовать.
- 404 Not Found — сервер не нашел запрашиваемый ресурс.
- 405 Method Not Allowed — метод запроса не поддерживается для данного ресурса.
- 418 I'm a teapot — я чайник. (код шутка RFC 2324)
- 500 Internal Server Error — внутренняя ошибка сервера.
- 502 Bad Gateway — сервер, действуя как шлюз, получил неверный ответ от вышестоящего сервера.
- 503 Service Unavailable — сервер временно не доступен.

In [8]:
ERROR_CODE = random.randint(400, 526)
print(f"Используемый код ошибки: {ERROR_CODE}")

Используемый код ошибки: 441


In [9]:
@app.get("/error")
async def generate_error():
    raise HTTPException(
        status_code=ERROR_CODE,
        detail=f"Тестовая ошибка с кодом {ERROR_CODE}. Я сервер - у меня лапки :)"
    )

In [10]:
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8002)

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
time.sleep(2)

base_url = "http://localhost:8002"

response = requests.get(f"{base_url}/error")
print(f"Статус ошибки: {response.status_code}")

INFO:     Started server process [60831]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8002 (Press CTRL+C to quit)


INFO:     127.0.0.1:38866 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:38870 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:38884 - "GET /error HTTP/1.1" 441 
Статус ошибки: 441


### Задание 4

Создание автодокументации для API: FastAPI автоматически генерирует документацию API в формате OpenAPI и предоставляет интерфейс Swagger UI для ее просмотра

*Поскольку код коллаба выполняется внутри виртуальной машины, у которой нет внешнего IP-адреса, вам потребуется создать тоннель, чтобы получить внешний IP-адрес.*

*Зарегистрируйтесь в личном кабинете https://xtunnel.ru/ и скопируйте бесплатную лицензию (секретный ключ API)*

Если возникают сложности, используйте локальную версию коллаба.
```bash
pip install notebook
jupyter notebook
```


Задача. Чтобы проверить документацию, выполните следующие шаги:
1. Запустите сервер FastAPI с помощью команды ниже.
2. Откройте браузер и перейдите по адресу `http://127.0.0.1:8000/docs` для просмотра документации в Swagger UI.
3. Для просмотра документации в формате OpenAPI перейдите по адресу `http://127.0.0.1:8000/openapi.json`

In [11]:
"""

вставьте сюда полученый JSON, который вы скопировали по адресу http://127.0.0.1:8000/openapi.json

"""

json_string = '{"openapi":"3.1.0","info":{"title":"FastAPI","version":"0.1.0"},"paths":{"/items/":{"post":{"summary":"Create Item","operationId":"create_item_items__post","requestBody":{"content":{"application/json":{"schema":{"$ref":"#/components/schemas/Item"}}},"required":true},"responses":{"200":{"description":"Successful Response","content":{"application/json":{"schema":{"additionalProperties":true,"type":"object","title":"Response Create Item Items  Post"}}}},"422":{"description":"Validation Error","content":{"application/json":{"schema":{"$ref":"#/components/schemas/HTTPValidationError"}}}}}}},"/items/{item_id}":{"get":{"summary":"Read Item","operationId":"read_item_items__item_id__get","parameters":[{"name":"item_id","in":"path","required":true,"schema":{"type":"integer","title":"Item Id"}}],"responses":{"200":{"description":"Successful Response","content":{"application/json":{"schema":{"type":"object","additionalProperties":true,"title":"Response Read Item Items  Item Id  Get"}}}},"422":{"description":"Validation Error","content":{"application/json":{"schema":{"$ref":"#/components/schemas/HTTPValidationError"}}}}}},"put":{"summary":"Update Item","operationId":"update_item_items__item_id__put","parameters":[{"name":"item_id","in":"path","required":true,"schema":{"type":"integer","title":"Item Id"}}],"requestBody":{"required":true,"content":{"application/json":{"schema":{"$ref":"#/components/schemas/Item"}}}},"responses":{"200":{"description":"Successful Response","content":{"application/json":{"schema":{"type":"object","additionalProperties":true,"title":"Response Update Item Items  Item Id  Put"}}}},"422":{"description":"Validation Error","content":{"application/json":{"schema":{"$ref":"#/components/schemas/HTTPValidationError"}}}}}},"delete":{"summary":"Delete Item","operationId":"delete_item_items__item_id__delete","parameters":[{"name":"item_id","in":"path","required":true,"schema":{"type":"integer","title":"Item Id"}}],"responses":{"200":{"description":"Successful Response","content":{"application/json":{"schema":{"type":"object","additionalProperties":true,"title":"Response Delete Item Items  Item Id  Delete"}}}},"422":{"description":"Validation Error","content":{"application/json":{"schema":{"$ref":"#/components/schemas/HTTPValidationError"}}}}}}},"/json_data":{"get":{"summary":"Get Json Data","operationId":"get_json_data_json_data_get","responses":{"200":{"description":"Successful Response","content":{"application/json":{"schema":{}}}}}}},"/error":{"get":{"summary":"Generate Error","operationId":"generate_error_error_get","responses":{"200":{"description":"Successful Response","content":{"application/json":{"schema":{}}}}}}}},"components":{"schemas":{"HTTPValidationError":{"properties":{"detail":{"items":{"$ref":"#/components/schemas/ValidationError"},"type":"array","title":"Detail"}},"type":"object","title":"HTTPValidationError"},"Item":{"properties":{"name":{"type":"string","title":"Name"},"description":{"anyOf":[{"type":"string"},{"type":"null"}],"title":"Description"}},"type":"object","required":["name"],"title":"Item"},"ValidationError":{"properties":{"loc":{"items":{"anyOf":[{"type":"string"},{"type":"integer"}]},"type":"array","title":"Location"},"msg":{"type":"string","title":"Message"},"type":{"type":"string","title":"Error Type"}},"type":"object","required":["loc","msg","type"],"title":"ValidationError"}}}}'
parsed_json = json.loads(json_string)
formatted_json = json.dumps(parsed_json, indent=4, ensure_ascii=False)
print(formatted_json)

{
    "openapi": "3.1.0",
    "info": {
        "title": "FastAPI",
        "version": "0.1.0"
    },
    "paths": {
        "/items/": {
            "post": {
                "summary": "Create Item",
                "operationId": "create_item_items__post",
                "requestBody": {
                    "content": {
                        "application/json": {
                            "schema": {
                                "$ref": "#/components/schemas/Item"
                            }
                        }
                    },
                    "required": true
                },
                "responses": {
                    "200": {
                        "description": "Successful Response",
                        "content": {
                            "application/json": {
                                "schema": {
                                    "additionalProperties": true,
                                    "type": "object",
            

In [12]:
base_url = "http://localhost:8002"

endpoints = [
    "/docs",
    "/redoc",
    "/openapi.json",
    "/json_data",
    "/items/",
    "/error"
]

print("Эндпоинты:")

for endpoint in endpoints:
    if endpoint == "/items/":
        response = requests.post(f"{base_url}{endpoint}", json={"name": "test", "description": "test item"})
    else:
        response = requests.get(f"{base_url}{endpoint}")

    print(f"{endpoint:20} | Status: {response.status_code:3} | Type: {response.headers.get('content-type', 'N/A')}")

Эндпоинты:
INFO:     127.0.0.1:38894 - "GET /docs HTTP/1.1" 200 OK
/docs                | Status: 200 | Type: text/html; charset=utf-8
INFO:     127.0.0.1:38896 - "GET /redoc HTTP/1.1" 200 OK
/redoc               | Status: 200 | Type: text/html; charset=utf-8
INFO:     127.0.0.1:38904 - "GET /openapi.json HTTP/1.1" 200 OK
/openapi.json        | Status: 200 | Type: application/json
INFO:     127.0.0.1:38918 - "GET /json_data HTTP/1.1" 200 OK
/json_data           | Status: 200 | Type: application/json
INFO:     127.0.0.1:38932 - "POST /items/ HTTP/1.1" 200 OK
/items/              | Status: 200 | Type: application/json
INFO:     127.0.0.1:38946 - "GET /error HTTP/1.1" 441 
/error               | Status: 441 | Type: application/json


In [13]:
from google.colab.output import eval_js
from IPython.display import display, IFrame, HTML

public_url = eval_js("google.colab.kernel.proxyPort(8002)")
swagger_url = f"{public_url}/docs"

# Создаем HTML с iframe и обработкой ошибок
html_code = f'''
<div style="border: 2px solid #e0e0e0; border-radius: 5px; padding: 10px;">
    <h3>FastAPI Swagger Documentation</h3>
    <p>Если документация не загружается, откройте ссылки ниже:</p>
    <iframe src="{swagger_url}" width="100%" height="600px" style="border: 1px solid #ccc; border-radius: 5px;">
        Ваш браузер не поддерживает iframe.
        <a href="{swagger_url}" target="_blank">Открыть документацию в новой вкладке</a>
    </iframe>
    <div style="margin-top: 10px;">
        <strong>Альтернативные ссылки:</strong><br>
        • <a href="{swagger_url}" target="_blank">Swagger UI</a><br>
        • <a href="{public_url}/redoc" target="_blank">ReDoc</a><br>
        • <a href="{public_url}/openapi.json" target="_blank">OpenAPI JSON</a>
    </div>
</div>
'''

display(HTML(html_code))

### Задание 5

Автодокументаци API хороша для маленьких проектов. Сейчас вам нужно полученый в задании 4 JSON вставить в редактор [Swagger](https://editor.swagger.io/), добавить проектируемый маршрут с ответом в формате XML и сохранить описание в YAML.

Задачи:
1. Откройте редактор Swagger, вставьте JSON (ответьте ОК на запрос Would you like to convert your JSON into YAML?)
2. Добавьте 1 эндпоинт.
3. Скопируйте получившееся описание в YAML и вставьте в ячейку ниже.

In [14]:
from IPython.display import display, Code

In [15]:
"""

вставьте сюда полученый YAML, который вы скопировали из SWagger

"""

yaml_string = """openapi: 3.1.0
info:
  title: FastAPI
  version: 0.1.0
paths:
  /items/:
    post:
      summary: Create Item
      operationId: create_item_items__post
      requestBody:
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/Item'
        required: true
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema:
                additionalProperties: true
                type: object
                title: Response Create Item Items  Post
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
  /items/{item_id}:
    get:
      summary: Read Item
      operationId: read_item_items__item_id__get
      parameters:
        - name: item_id
          in: path
          required: true
          schema:
            type: integer
            title: Item Id
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema:
                type: object
                additionalProperties: true
                title: Response Read Item Items  Item Id  Get
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
    put:
      summary: Update Item
      operationId: update_item_items__item_id__put
      parameters:
        - name: item_id
          in: path
          required: true
          schema:
            type: integer
            title: Item Id
      requestBody:
        required: true
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/Item'
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema:
                type: object
                additionalProperties: true
                title: Response Update Item Items  Item Id  Put
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
    delete:
      summary: Delete Item
      operationId: delete_item_items__item_id__delete
      parameters:
        - name: item_id
          in: path
          required: true
          schema:
            type: integer
            title: Item Id
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema:
                type: object
                additionalProperties: true
                title: Response Delete Item Items  Item Id  Delete
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
  /json_data:
    get:
      summary: Get Json Data
      operationId: get_json_data_json_data_get
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema: {}
  /error:
    get:
      summary: Generate Error
      operationId: generate_error_error_get
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema: {}
components:
  schemas:
    HTTPValidationError:
      properties:
        detail:
          items:
            $ref: '#/components/schemas/ValidationError'
          type: array
          title: Detail
      type: object
      title: HTTPValidationError
    Item:
      properties:
        name:
          type: string
          title: Name
        description:
          anyOf:
            - type: string
            - type: string
          title: Description
      type: object
      required:
        - name
      title: Item
    ValidationError:
      properties:
        loc:
          items:
            anyOf:
              - type: string
              - type: integer
          type: array
          title: Location
        msg:
          type: string
          title: Message
        type:
          type: string
          title: Error Type
      type: object
      required:
        - loc
"""

display(Code(yaml_string, language='yaml'))

openapi: 3.1.0
info:
  title: FastAPI
  version: 0.1.0
paths:
  /items/:
    post:
      summary: Create Item
      operationId: create_item_items__post
      requestBody:
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/Item'
        required: true
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema:
                additionalProperties: true
                type: object
                title: Response Create Item Items  Post
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
  /items/{item_id}:
    get:
      summary: Read Item
      operationId: read_item_items__item_id__get
      parameters:
        - name: item_id
          in: path
          required: true
          schema:
            type: integer
            title: Item Id
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema:
                type: object
                additionalProperties: true
                title: Response Read Item Items  Item Id  Get
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
    put:
      summary: Update Item
      operationId: update_item_items__item_id__put
      parameters:
        - name: item_id
          in: path
          required: true
          schema:
            type: integer
            title: Item Id
      requestBody:
        required: true
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/Item'
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema:
                type: object
                additionalProperties: true
                title: Response Update Item Items  Item Id  Put
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
    delete:
      summary: Delete Item
      operationId: delete_item_items__item_id__delete
      parameters:
        - name: item_id
          in: path
          required: true
          schema:
            type: integer
            title: Item Id
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema:
                type: object
                additionalProperties: true
                title: Response Delete Item Items  Item Id  Delete
        '422':
          description: Validation Error
          content:
            application/json:
              schema:
                $ref: '#/components/schemas/HTTPValidationError'
  /json_data:
    get:
      summary: Get Json Data
      operationId: get_json_data_json_data_get
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema: {}
  /error:
    get:
      summary: Generate Error
      operationId: generate_error_error_get
      responses:
        '200':
          description: Successful Response
          content:
            application/json:
              schema: {}
components:
  schemas:
    HTTPValidationError:
      properties:
        detail:
          items:
            $ref: '#/components/schemas/ValidationError'
          type: array
          title: Detail
      type: object
      title: HTTPValidationError
    Item:
      properties:
        name:
          type: string
          title: Name
        description:
          anyOf:
            - type: string
  

## Итоговое оформление


1. Подготовьте ноутбук в логичной структуре: написание кода → работа с JSON → обработка ошибок → API в YAML → итоги.  
2. В ячейках Markdown сформулируйте 5–8 предложений с выводами, когда стоит применять подход Code-first и почему стоит придерживаться подхода API-first.  



Code-first уместен для одиночных разработчиках в быстрых прототипах и очень мелких проектах, когда важна скорость. Подход - «сначала написать, потом описать». Этот подход хорош при преимущественно внутреннем использовании небольшого сервиса, при тривиальных реализациях в некритичных применениях.
<br>API-first предпочтителен, когда над продуктом работают несколько разрабочиков, распределённые команды требуется согласование контрактов до разработки, чтобы снизить риски расхождений и дорогостоящего рефакторинга. Применение API-first позволяет параллелить работу фронтенда и бэкенда, генерировать SDK/клиенты/валидаторы и ускорять интеграции. API-first делает требования прозрачными, упрощает тестирование, версионирование и управление совместимостью, а также повышает качество документации. В долгосрочной перспективе API-first снижает риски и издержки.